In [1]:
# full_inventory_policy.py

import pandas as pd
import numpy as np
from scipy.stats import weibull_min

# -----------------------------
# 1. Reliability fitting
# -----------------------------
def fit_weibull(failures, part_id):
    df = failures[failures["part_id"] == part_id].sort_values("failure_date")
    if len(df) < 5:
        return None
    inter = df["failure_date"].diff().dt.days.dropna().values
    shape, loc, scale = weibull_min.fit(inter, floc=0)
    mtbf = inter.mean()
    return shape, scale, mtbf

# -----------------------------
# 2. Croston’s Method
# -----------------------------
def croston(ts, alpha=0.1, h=12):
    ts = np.array(ts)
    z, p, q = 0.0, 1.0, 0.0
    forecast = []
    for i in range(len(ts)):
        if ts[i] > 0:
            z = alpha*ts[i] + (1-alpha)*z
            p = alpha*(i+1) + (1-alpha)*p
            q = z/max(1,p)
        forecast.append(q)
    return forecast[-1] * np.ones(h)

# -----------------------------
# 3. Safety Stock
# -----------------------------
def safety_stock(daily_demand, lead_time, service_level=0.95):
    z_table = {0.90:1.28, 0.95:1.645, 0.99:2.33}
    z = z_table.get(service_level, 1.645)
    sigma = np.std(daily_demand)
    return z * sigma * np.sqrt(lead_time)

# -----------------------------
# 4. ABC–VED Classification
# -----------------------------
def abc_analysis(parts, policy):
    val = policy[["part_id","muL"]].groupby("part_id").sum().reset_index()
    val = val.merge(parts[["part_id","unit_cost"]], on="part_id", how="left")
    val["demand_value"] = val["muL"] * val["unit_cost"]
    val = val.sort_values("demand_value", ascending=False).reset_index(drop=True)

    total = val["demand_value"].sum()
    val["cum_share"] = val["demand_value"].cumsum() / total
    val["ABC"] = np.where(val["cum_share"] <= 0.80, "A",
                   np.where(val["cum_share"] <= 0.95, "B", "C"))
    return val[["part_id","ABC","demand_value","cum_share"]]

def ved_analysis(parts, policy):
    rate = policy.groupby("part_id", as_index=False)["daily_rate"].sum().rename(columns={"daily_rate":"rate"})
    feat = parts[["part_id","unit_cost","lead_time_days"]].merge(rate, on="part_id", how="left").fillna(0)

    def _norm(x):
        return (x - x.min()) / (x.max() - x.min()) if x.max() > x.min() else np.zeros(len(x))

    feat["lt_n"]   = _norm(feat["lead_time_days"])
    feat["cost_n"] = _norm(feat["unit_cost"])
    feat["rate_n"] = _norm(feat["rate"])

    feat["severity_score"] = 0.45*feat["lt_n"] + 0.35*feat["cost_n"] + 0.20*feat["rate_n"]
    feat["VED"] = np.where(feat["severity_score"] >= 0.66, "V",
                    np.where(feat["severity_score"] >= 0.33, "E", "D"))
    return feat[["part_id","VED","severity_score"]]

# -----------------------------
# 5. Batch Process
# -----------------------------
def generate_full_policy(failures, orders, parts, policy, horizon=30, service_level=0.95):
    results = []
    for part in parts["part_id"]:
        rel = fit_weibull(failures, part)
        if rel is None:
            continue
        shape, scale, mtbf = rel

        # demand series
        od = orders[orders["part_id"] == part]
        if od.empty:
            continue
        daily = od.groupby("request_date")["qty"].sum().reindex(
            pd.date_range(od["request_date"].min(), od["request_date"].max()), fill_value=0
        )

        forecast = croston(daily.values, h=horizon).mean()
        lead_time = int(parts.loc[parts["part_id"]==part,"lead_time_days"].values[0])
        ss = safety_stock(daily.values, lead_time, service_level)

        results.append({
            "part_id": part,
            "weibull_shape": shape,
            "weibull_scale": scale,
            "MTBF": mtbf,
            "croston_forecast": forecast,
            "safety_stock": ss,
            "lead_time": lead_time
        })
    df = pd.DataFrame(results)

    # Merge with ABC–VED
    abc = abc_analysis(parts, policy)
    ved = ved_analysis(parts, policy)
    df = df.merge(abc, on="part_id", how="left").merge(ved, on="part_id", how="left")
    df["ABC_VED"] = df["ABC"].fillna("C") + "-" + df["VED"].fillna("D")
    return df

# -----------------------------
# 6. Example Usage
# -----------------------------
if __name__ == "__main__":
    failures = pd.read_csv("failures (1).csv", parse_dates=["failure_date"])
    orders   = pd.read_csv("service_orders (1).csv", parse_dates=["request_date"])
    parts    = pd.read_csv("parts_master (1).csv")
    policy   = pd.read_csv("policy_base_stock (1).csv")

    full_policy = generate_full_policy(failures, orders, parts, policy, horizon=30, service_level=0.95)
    print(full_policy.head())

    full_policy.to_csv("policy_full_reliability_demand_ABC_VED.csv", index=False)
    print("\n✅ Saved → policy_full_reliability_demand_ABC_VED.csv")


  part_id  weibull_shape  weibull_scale      MTBF  croston_forecast  \
0    P001       1.270964       3.225313  2.803468          0.022630   
1    P002       1.335012       2.543683  2.124891          0.032975   
2    P003       1.247368       8.118323  7.345455          0.010201   
3    P004       1.117905       8.992921  8.494737          0.008985   
4    P005       1.204389       4.904029  4.499076          0.015040   

   safety_stock  lead_time ABC   demand_value  cum_share VED  severity_score  \
0      8.227477          7   C    2177.598192   0.999701   D        0.126873   
1     26.243185         56   A  510461.304848   0.505438   V        0.710813   
2     10.024607         27   A  246165.811011   0.775787   E        0.545527   
3      9.807440         25   B   92592.686935   0.947087   E        0.335733   
4     14.956509         33   A  301508.336894   0.710758   E        0.499889   

  ABC_VED  
0     C-D  
1     A-V  
2     A-E  
3     B-E  
4     A-E  

✅ Saved → policy_fu